# Correlation between the number of positive Covid cases/ temperature in the USA

In [1]:
%matplotlib inline

# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.dates as mdates
import time
import scipy.stats as st
from scipy.stats import linregress
import requests
from collections import defaultdict
import json

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
#Reading CSV
positivecases_tests_df = pd.read_csv("Resources/positivecases_tests.csv")
positivecases_tests_df.head()

,date,US Population,US Deaths,US Positive,US PositiveIncrease,US TotalTestResults,US TotalTestResultsIncrease,NC Population,NC death,NC Positive,NC PositiveIncrease,NC TotalTestResults,NC TotalTestResultsIncrease,NC_positive_cases,USA_positive_cases
0,2021-03-07,332031554,515151.0,28756489.0,41835,363825123,1170059,10565885.0,11502.0,872176.0,0.0,9688838.0,0.0,8254.642181,8660.769934
1,2021-03-06,332031554,514309.0,28714654.0,60015,362655064,1430992,10565885.0,11502.0,872176.0,2027.0,9688838.0,38654.0,8254.642181,8648.170228
2,2021-03-05,332031554,512629.0,28654639.0,68787,361224072,1744417,10565885.0,11446.0,870149.0,2093.0,9650184.0,44487.0,8235.457796,8630.095138
3,2021-03-04,332031554,510408.0,28585852.0,65487,359479655,1590984,10565885.0,11399.0,868056.0,2502.0,9605697.0,44050.0,8215.648760,8609.378132
4,2021-03-03,332031554,508665.0,28520365.0,66836,357888671,1406795,10565885.0,11363.0,865554.0,2145.0,9561647.0,19976.0,8191.968775,8589.655006


In [3]:
#Making sure the column "date" is in date format
positivecases_tests_df["date"]=pd.to_datetime(positivecases_tests_df["date"])

In [4]:
positivecases_tests_df["month_year"] = positivecases_tests_df["date"].dt.to_period("M")

In [5]:
NC_monthly_cases_df = positivecases_tests_df.groupby(["month_year"])["NC Positive"].sum().reset_index()
NC_monthly_cases_df

,month_year,NC Positive
0,2020-01,0.0
1,2020-02,0.0
2,2020-03,8293.0
3,2020-04,168564.0
4,2020-05,571848.0
5,2020-06,1375755.0
6,2020-07,2901556.0
7,2020-08,4499595.0
8,2020-09,5664512.0
9,2020-10,7494809.0


In [6]:
US_monthly_cases_df = positivecases_tests_df.groupby(["month_year"])["US Positive"].sum().reset_index()
US_monthly_cases_df

,month_year,US Positive
0,2020-01,24.0
1,2020-02,202.0
2,2020-03,1206666.0
3,2020-04,19610160.0
4,2020-05,45273840.0
5,2020-06,64577037.0
6,2020-07,110431975.0
7,2020-08,165003108.0
8,2020-09,197441612.0
9,2020-10,248879799.0


#### Using OpenWeatherMap API to get the average temperature by month/year to test the correlation "Positive Cases x Temperature"

In [7]:
# Creating a list of states or locations you want to get temperature for
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 
          'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [8]:
# Creating a dictionary to store the temperature data
data_2020 = defaultdict(lambda: defaultdict(dict))
data_2021 = defaultdict(lambda: defaultdict(dict))

In [43]:
#TEST TO CHECK THE RESPONSE NEEDED
x = {
"cod":200,
"city_id":5400075,
"calctime":3.105838634,
"result":[
   {
      "month": 1,
      "day": 1,
      "temp":{
         "record_min":274.44,
         "record_max":290.45,
         "average_min":276.79,
         "average_max":286.5,
         "median":281.26,
         "mean":281.29,
         "p25":278.57,
         "p75":283.83,
         "st_dev":3.69,
         "num":136
      },
      "pressure":{
         "min":1014,
         "max":1026,
         "median":1021,
         "mean":1020.5,
         "p25":1019,
         "p75":1023,
         "st_dev":3.25,
         "num":135
      },
      "humidity":{
         "min":23,
         "max":100,
         "median":70,
         "mean":65.88,
         "p25":47,
         "p75":82,
         "st_dev":20.63,
         "num":133
      },
      "wind":{
         "min":0,
         "max":10,
         "median":1,
         "mean":2.36,
         "p25":1,
         "p75":3,
         "st_dev":2.3,
         "num":136
      },
      "precipitation":{
         "min":0,
         "max":0.3,
         "median":0,
         "mean":0.01,
         "p25":0,
         "p75":0,
         "st_dev":0.04,
         "num":136
      },
      "clouds":{
         "min":0,
         "max":90,
         "median":1,
         "mean":20.18,
         "p25":1,
         "p75":40,
         "st_dev":31.91,
         "num":136
      }
   }
]
}

In [48]:
#TEST RESULT:
x['result'][0]['temp']['mean']

281.29

In [49]:
# Iterate over each state and make API request to fetch temperature
for state in states:
     # Make API requests for each month in 2020
        for month in range(1,13):
            #url_2020 = f"https://history.openweathermap.org/data/2.5/aggregated/month?q={state},US&year=2020&month={month}appid={weather_api_key}&units=metric"
            url_2020 = f"http://api.openweathermap.org/data/2.5/weather?q={state},US&appid={weather_api_key}&units=metric&dt=2020-{month}-01"
            response_2020 = requests.get(url_2020)
            data_2020 = response_2020.json()
            
            # Extract temperature from 2020 response
            try:
               # Parsing the response
                median_temp_2020 = data_2020['result'][0]['temp']['median']
                mean_temp_2020 = data_2020['result'][0]['temp']['mean']
                
                data_2020.append({"State": state,
                                  "Month": month,
                                  "Median Temperature": median_temp_2020,
                                  "Mean Temperature": mean_temp_2020})
            except KeyError:
                print(f"No temperature data available for {state}, 2020-{month}")
                continue  # Skip to the next iteration of the loop        
            
    

No temperature data available for AL, 2020-1
No temperature data available for AL, 2020-2
No temperature data available for AL, 2020-3
No temperature data available for AL, 2020-4
No temperature data available for AL, 2020-5
No temperature data available for AL, 2020-6
No temperature data available for AL, 2020-7
No temperature data available for AL, 2020-8
No temperature data available for AL, 2020-9
No temperature data available for AL, 2020-10
No temperature data available for AL, 2020-11
No temperature data available for AL, 2020-12
No temperature data available for AK, 2020-1
No temperature data available for AK, 2020-2
No temperature data available for AK, 2020-3
No temperature data available for AK, 2020-4
No temperature data available for AK, 2020-5
No temperature data available for AK, 2020-6
No temperature data available for AK, 2020-7
No temperature data available for AK, 2020-8
No temperature data available for AK, 2020-9
No temperature data available for AK, 2020-10
No tem

KeyboardInterrupt: 

In [50]:
#TEST TO CHECK THE ERROR:
requests.get(f'https://history.openweathermap.org/data/2.5/aggregated/year?q=London&appid=')

#The 401 (Unauthorized) status code indicates that the request 
#has not been applied because it lacks valid authentication 
#credentials for the target resource.

<Response [401]>

In [51]:
#TEST TO CHECK THE ERROR - INVALID KEY
requests.get(f'https://history.openweathermap.org/data/2.5/history/city?q=London&type=hour&start={start}&end={end}&appid={weather_api_key}').json()

{'cod': 401,
 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}

In [52]:
#CHECKING THE KEY - THE KEY WORKS FOR THE REGULAR API
requests.get(f'https://api.openweathermap.org/data/2.5/weather?q=London&appid=ee1f2632aa2a17d1485bbcd6a87d7247').json()

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 285.64,
  'feels_like': 285.16,
  'temp_min': 284.51,
  'temp_max': 286.91,
  'pressure': 1006,
  'humidity': 85},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 250},
 'clouds': {'all': 40},
 'dt': 1700415867,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1700378684,
  'sunset': 1700410030},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

## We were not able to get the historical temperatures average using the OpenWeather API. The key that allows us to obtain this information is paid(950USD/month)